上一节我们看到可以使用逻辑回归来求解分类问题：

$$
ln \frac{y}{1-y} = wx+b
$$

但是这里的 w 和 b 要怎么计算呢？前面在求解线性回归时，我们首先确定模型的损失函数，然后通过计算损失函数的最小值从而得到模型的各个参数。那么逻辑回归的损失函数该如何确定呢？

如果按照最小二乘的求解思路，这里的 $(x, ln \frac{y}{1-y})$ 呈线性关系，可以定义出损失函数：

$$
loss = \sum_{i=1}^{n}(ln \frac{y_i}{1-y_i} - (wx_i + b))^2
$$

但是很显然，$y_i$ 的取值不是 0 就是 1，代入上面的损失函数要么分母是 0 要么分子是 0，根本没法求解。回忆前面一节 S 型函数的拟合曲线，实际上，它的值域范围是 $(0, 1)$，并不是等于 0 或 1，只不过取值非常接近 0 或 1，我们不妨把这个值理解为可能性。

若将 y 视为样本 x 作为正例的可能性，则 1-y 则是 x 作为反例的可能性，两者的比值 $\frac{y}{1-y}$ 称为 **几率**（odds），反映了 x 作为正例的相对可能性。对几率取对数 $ln \frac{y}{1-y}$ 就是 **对数几率**（log odds，亦称 logit），这也是对数几率函数名称的由来，而使用对数几率函数拟合分类数据的这种回归方法被称为 **对数几率回归**（logistic regression），可能是中文比较拗口，很多地方喜欢把它称为 **逻辑回归**（logistic regression， 或者 **对数回归**）。不过要注意的是，虽然名字中有回归两个字，但它解决的是分类问题，它不仅能够预测“类别”，还能给出近似的概率预测。

这样我们就可以把 $y$ 改写成 $p(y = 1 | x)$，这是一个条件概率，表示当 $x$ 发生时 $y = 1$ 的概率，这个概率也被称为 **后验概率**，同理，我们把 $1-y$ 改写成 $p(y = 0 | x)$ 表示当 $x$ 发生时 $y = 0$ 的概率。所以有：

$$
ln \frac{p(y = 1 | x)}{p(y = 0 | x)} = wx+b
$$

并把分类问题改写成下面的条件概率形式：

$$
p(y = 1 | x) = \frac{e^{wx+b}}{1+e^{wx+b}}
$$

$$
p(y = 0 | x) = \frac{1}{1+e^{wx+b}}
$$

这就是逻辑回归模型。

### 极大似然法解逻辑回归

上面的逻辑回归模型和线性回归模型很不一样，没办法再用最小二乘来进行参数估计了。在最小二乘中，我们有每个样本的实际值和它对应的预测值，我们可以计算出两者的平方损失，然后累加起来，最小二乘的目标是使得累加起来的损失最小。而在这里，我们有每个样本的实际值和它对应的预测值的概率，如果我们的模型能让每个样本都正确分类，这样的模型损失是最小的，要实现这一点，也就是说我们要找一个模型让每个样本属于其真实分类的概率越大越好，这种方法被称为 **极大似然法**（maximum likelihood method）。

$$
\mathop{\arg \max}_{w,b} \prod_{i=1}^n p(y_i | x_i; w,b)
$$

但是这里的连乘结果很可能导致下溢，通常使用**对数似然**（log-likelihood）来代替，这样就可以将连乘转换成连加：

$$
\mathop{\arg \max}_{w,b} \sum_{i=1}^n \ln p(y_i | x_i; w,b)
$$

其中 $p(y | x)$ 包含了 $y = 0$ 和 $y = 1$ 两种情况，这里有一个小技巧可以把两种情况写成一个表达式：

$$
p(y|x) = p(y=1|x)^y p(y=0|x)^{(1-y)}
$$

对其求对数有：

$$
\ln p(y|x) = y \ln p(y=1|x) + (1-y) \ln p(y=0|x)
$$

令：$p(y=1|x) = \pi(x)$，那么有 $p(y=0|x) = 1 - \pi(x)$，所以有：

$$
\begin{align}
\ln p(y|x) &= y \ln \pi(x) + (1-y) \ln (1-\pi(x)) \\
&= y \ln \pi(x) + \ln (1-\pi(x)) - y\ln(1-\pi(x)) \\
&= y \ln \frac{\pi(x)}{1-\pi(x)} + \ln (1-\pi(x)) \\
&= y \ln e^{wx+b} + \ln \frac{1}{1+e^{wx+b}} \\
&= y(wx+b) - \ln (1+e^{wx+b})
\end{align}
$$

于是我们得到了逻辑回归的损失函数：

$$
\begin{align}
loss 
&= \sum_{i=1}^n \ln p(y_i|x_i) \\
&= \sum_{i=1}^n y_i \ln \pi(x_i) + (1-y_i) \ln (1-\pi(x_i)) \\
&= \sum_{i=1}^n [ y_i(wx_i+b) - \ln (1+e^{wx_i+b}) ] \\
\end{align}
$$

我们要求的就是上式最大值时的 w 和 b 参数，一般情况下，我们都是求损失函数的最小值，所以我们在上式中加一个负号。

$$
\mathop{\arg \min}_{w,b} \sum_{i=1}^n [ -y_i(wx_i+b) + \ln (1+e^{wx_i+b}) ]
$$

### 梯度下降法

逻辑回归的损失函数不能像线性回归那样求解析解，一般我们通过梯度下降法或牛顿法等优化算法来求解。这里我们使用梯度下降法，首先我们需要求出上面的损失函数的梯度，也就是分别对 w 和 b 求偏导。

这里要运用到微分法则中很重要的一条：**链式法则**，即：

$$
\frac{\partial y}{\partial x} = \frac{\partial y}{\partial z} \frac{\partial z}{\partial x}
$$

我们令：

$$
z = \ln p = \ln (1+e^{wx_i+b})
$$

则：

$$
\frac{\partial z}{\partial w} = \frac{\partial z}{\partial p} \frac{\partial p}{\partial w} = \frac{1}{p} e^{wx_i+b} x_i = \frac{e^{wx_i+b}}{1+e^{wx_i+b}}x_i
$$

$$
\frac{\partial z}{\partial b} = \frac{\partial z}{\partial p} \frac{\partial p}{\partial b} = \frac{1}{p} e^{wx_i+b} = \frac{e^{wx_i+b}}{1+e^{wx_i+b}}
$$

再令：

$$
z = -y_i(wx_i+b)
$$

则：

$$
\frac{\partial z}{\partial w} = -y_ix_i
$$

$$
\frac{\partial z}{\partial b} = -y_i
$$

于是有：

$$
\begin{align}
\frac{\partial loss}{\partial w} &= \sum_{i=1}^n (\frac{e^{wx_i+b}}{1+e^{wx_i+b}}x_i -y_ix_i) \\
&= \sum_{i=1}^n (\frac{e^{wx_i+b}}{1+e^{wx_i+b}} -y_i)x_i \\
&= \sum_{i=1}^n (\pi(x_i) -y_i)x_i
\end{align}
$$

$$
\begin{align}
\frac{\partial loss}{\partial b} &= \sum_{i=1}^n (\frac{e^{wx_i+b}}{1+e^{wx_i+b}} -y_i) \\
&= \sum_{i=1}^n (\pi(x_i) -y_i)
\end{align}
$$

// TODO 梯度下降算法

// TODO 牛顿法

// 一元分类 -> 二元分类 -> 多元分类
// 二分类 -> 多分类
// 线性分类 -> 非线性分类